In [1]:
import torch 
from torch import nn
from torch.nn import init
import torch.utils.data as data_utils
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
SEED = 2021
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [2]:
dataset = np.loadtxt(fname='E:/data/Abilene.txt', delimiter="\t", skiprows=1)
N_ORIGIN = int(np.max(dataset[:,0]))
N_DESTINATION = int(np.max(dataset[:,1]))
N_TIME = int(np.max(dataset[:,2]))
EMB_SIZE = 64
NEG_WEIGHT = 0.1
DROP_RATIO = 0.3
LEARNING_RATE = 0.05
BATCH_SIZE = 128
EPOCH = 200
#train_index = np.random.choice(list(range(dataset.shape[0])), size = int(0.3*dataset.shape[0]), replace=False)
#test_index = list(set(range(dataset.shape[0])) - set(train_index))

In [3]:
flag = np.inf
for i in range(1,N_TIME):
    tmp = len(dataset[dataset[:,2]==i]) / (N_ORIGIN*N_DESTINATION)
    if tmp < flag:
        flag = tmp
print(flag)

0.6805555555555556


In [6]:
len(dataset) / (N_ORIGIN*N_DESTINATION*N_TIME)

0.9493114688969724

In [3]:
def generate_train_from_local(path, index, n_origin, n_destination, n_time):
    data = np.loadtxt(fname=path, delimiter="\t", skiprows=1)[index,:]
    data[:,:3] = data[:,:3] - 1 
    train_tensor = np.zeros((n_origin, n_destination, n_time))
    for line in data:
        train_tensor[int(line[0]),int(line[1]),int(line[2])] = line[3]
    pos = dict()
    odt = dict()
    max_time_id = train_tensor.shape[2]
    max_time_num = 0
    for o in range(train_tensor.shape[0]):
        for d in range(train_tensor.shape[1]):
            pos_time = list(np.nonzero(train_tensor[o][d])[0])
            pos_time_num = len(pos_time)
            if  pos_time_num > max_time_num:
                max_time_num = pos_time_num
            pos[(o,d)] = pos_time
            odt[(o,d)] = list(train_tensor[o][d][pos_time])
    train_od = list()
    train_y = list()
    train_time = list()
    for i, j in pos.keys():
        while len(pos[(i,j)]) < max_time_num:
            pos[(i,j)].append(max_time_num)
            odt[(i,j)].append(0)
        train_od.append([i,j])
        train_time.append(pos[(i,j)])
        train_y.append(odt[(i,j)])
    return np.array(train_od), np.array(train_time), np.array(train_y)

def generate_test_from_local(path, index):
    data = np.loadtxt(fname=path, delimiter="\t", skiprows=1)[index,:]
    data[:,:3] = data[:,:3] - 1 
    return np.array(data[:,:2]), np.array(data[:,2]), data[:,3]

train_od, train_time, train_y = generate_train_from_local(path="E:/data/Abilene.txt", index = train_index,
                                                               n_origin=N_ORIGIN, n_destination=N_DESTINATION, n_time=N_TIME)
test_od, test_time, test_y = generate_test_from_local(path="E:/data/Abilene.txt", index = test_index)

In [4]:
class ENMF(nn.Module):
    def __init__(self, emb_size, n_origin, n_destination, n_time, neg_weight, drop_out):
        super().__init__()
        self.n_origin = n_origin
        self.n_destination = n_destination
        self.n_time = n_time
        self.neg_weight = neg_weight
        self.emb_size   = emb_size
        self.origin_embs = nn.Embedding(n_origin, emb_size)
        self.destination_embs = nn.Embedding(n_destination, emb_size)
        self.time_embs = nn.Embedding(n_time+1, emb_size)
        self.h = nn.Parameter(torch.randn(emb_size, 1))
        self.dropout = nn.Dropout(p=drop_out)
        self._reset_para()
        return
    
    def _reset_para(self):
        nn.init.xavier_normal_(self.origin_embs.weight)
        nn.init.xavier_normal_(self.destination_embs.weight)
        nn.init.xavier_normal_(self.time_embs.weight)
        nn.init.constant_(self.h, 0.01)
        return
    
    def forward(self, oids, dids, pos_tids, pos_value):
        '''
        uids: B
        dids: B
        pos_tids: B * L
        pos_value: B * L
        '''
        o_emb = self.dropout(self.origin_embs(oids))
        d_emb = self.dropout(self.destination_embs(dids))
        pos_embs = self.time_embs(pos_tids)

        # torch.einsum("ab,abc->abc")
        # B * L * D
        mask = (~(pos_tids.eq(self.n_time))).float()
        pos_embs = pos_embs * mask.unsqueeze(2)
        
        #pos_value = pos_value * ((~(pos_value.eq(-1))).float()).unsqueeze(2)
        
        # torch.einsum("ac,abc->abc")
        # B * L * D
        odt = o_emb.unsqueeze(1) * d_emb.unsqueeze(1) * pos_embs
        # torch.einsum("ajk,kl->ajl")
        # B * L
        hodt = odt.matmul(self.h).squeeze(2)

        # loss
        #print(odt.size())
        #print(hodt.size())
        #print(pos_value.size())
        pos_data_loss = torch.sum((1 - self.neg_weight) * hodt.square() - 2.0 * hodt * pos_value) # 添加真实值

        # torch.einsum("ab,ac->abc")
        part_1 = self.time_embs.weight.unsqueeze(2).bmm(self.time_embs.weight.unsqueeze(1))
        part_2 = o_emb.unsqueeze(2).bmm(o_emb.unsqueeze(1))
        part_3 = d_emb.unsqueeze(2).bmm(d_emb.unsqueeze(1))
        
        # D * D
        part_1 = part_1.sum(0)
        part_2 = part_2.sum(0)
        part_3 = part_3.sum(0)
        part_4 = self.h.mm(self.h.t())
        all_data_loss = torch.sum(part_1 * part_2 * part_3 * part_4)

        loss = self.neg_weight * all_data_loss + pos_data_loss
        return loss
    
    def predict(self, oid, did):
        '''
        oid: B
        did: B
        '''
        oid_embs = self.origin_embs(oid)
        did_embs = self.destination_embs(did)
        origin_destination_all_times = oid_embs.unsqueeze(1) * did_embs.unsqueeze(1) * self.time_embs.weight
        pred = origin_destination_all_times.matmul(self.h).squeeze(2)
        return torch.relu(pred)
    
'''    def rank(self, user):
        res = self.user_embs(user).unsqueeze(0)
        res = res * self.item_embs.weight
        res = res.matmul(self.h).squeeze(1)
        return res'''

'    def rank(self, user):\n        res = self.user_embs(user).unsqueeze(0)\n        res = res * self.item_embs.weight\n        res = res.matmul(self.h).squeeze(1)\n        return res'

In [5]:
def createLoader(train_od, train_time, train_y, test_od, test_time, test_y, batch_size):
    torch_x1 = torch.from_numpy(train_od).type(torch.LongTensor)
    torch_x2 = torch.from_numpy(train_time).type(torch.LongTensor)
    torch_y  = torch.from_numpy(train_y).type(torch.FloatTensor)
    torch_test1 = torch.from_numpy(test_od).type(torch.LongTensor)
    torch_test2 = torch.from_numpy(test_time).type(torch.LongTensor)
    torch_test3 = torch.from_numpy(test_y).type(torch.FloatTensor)
    torch_dataset = data_utils.TensorDataset(torch_x1, torch_x2, torch_y)
    train_loader = data_utils.DataLoader(dataset = torch_dataset, batch_size = batch_size, shuffle = True, num_workers = 0)
    torch_testset = data_utils.TensorDataset(torch_test1[:,0], torch_test1[:,1], torch_test2, torch_test3)
    test_loader = data_utils.DataLoader(dataset = torch_testset, batch_size = batch_size, num_workers = 0)
    return train_loader, test_loader

def createModel(emb_size, lr, n_origin, n_destination, n_time, neg_weight, drop_out):
    model = ENMF(emb_size=emb_size, n_origin=n_origin, n_destination=n_destination, n_time=n_time, 
                 neg_weight=neg_weight, drop_out = drop_out)
    if(torch.cuda.is_available()):
        model = model.cuda()
    optimizer = torch.optim.Adagrad(model.parameters(), lr = lr)
    print(model)
    return model, optimizer

In [6]:
def eval_1(model, test_loader):
    hit_list = list()
    undcg_list = list()
    rank_all_users = list()
    model.eval()
    with torch.no_grad(): 
        rmse_tmp = mae_tmp = ter_tmp = 0
        count = 0
        for step, (batch_o, batch_d, batch_t, batch_y) in enumerate(test_loader):
            if torch.cuda.is_available():
                batch_o, batch_d = batch_o.cuda(), batch_d.cuda()
            prediction = model.predict(batch_o, batch_d)
            pred_vector = prediction.cpu().data.numpy()
            for i, vec in enumerate(pred_vector):
                real = batch_y[i]
                res  = vec[batch_t[i]]
                rmse_tmp += torch.pow((res - real), 2).numpy()
                mae_tmp += torch.abs(res - real).numpy()
                ter_tmp += torch.pow(real, 2).numpy()
                count += 1
        rmse = np.sqrt(rmse_tmp / count)
        mae  = mae_tmp / count
        ter  = np.sqrt(rmse_tmp) / np.sqrt(ter_tmp)
        print(np.sqrt(rmse_tmp))
        print(np.sqrt(ter_tmp))
    model.train()
    print('RMSE:', rmse, ' ≈ %.6f' %  rmse)
    print('MAE :', mae,  ' ≈ %.6f' %  mae)
    print('TER :', ter,  ' ≈ %.6f' %  ter)
    return rmse, mae, ter

In [7]:
def train(train_od, train_time,train_y, test_od, test_time, test_y, epoch, batch_size, emb_size, lr, drop_out, neg_weight, n_origin, n_destination, n_time): 
    train_loader, test_loader = createLoader(train_od, train_time, train_y, test_od, test_time, test_y, batch_size)
    model, optimizer = createModel(emb_size, lr, n_origin, n_destination, n_time, neg_weight, drop_out)
    train_loss_list = list()
    rmse_list = [.0]
    mae_list = [.0]
    ter_list = [.0]
    model.train()
    for e in range(epoch):
        train_loss = list()
        for step, (batch_x1, batch_x2, batch_y) in enumerate(train_loader):
            x1, x2, x3, y = batch_x1[:,0], batch_x1[:,1], batch_x2, batch_y
            if (torch.cuda.is_available()):
                x1, x2, x3, y = x1.cuda(), x2.cuda(), x3.cuda(), y.cuda()
            optimizer.zero_grad()
            loss = model(x1, x2, x3, y)
            loss.backward()        
            train_loss.append(loss.cpu().item())
            optimizer.step()
        print('------第'+str(e+1)+'个epoch------')
        mean_train_loss = np.mean(train_loss)
        print('train_loss:', mean_train_loss)
        train_loss_list.append(mean_train_loss)
        if (e+1) % 200 == 0:
            rmse, mae, ter = eval_1(model, test_loader)
            rmse_list.append(rmse)
            mae_list.append(mae)
            ter_list.append(ter)
    #np.savetxt("./evalres/enmf/train_loss_list_"+str(epoch)+"epoch.txt", train_loss_list)    
    #np.savetxt("./evalres/enmf/rmse_list_"+str(epoch)+"epoch.txt", rmse_list)
    torch.cuda.empty_cache()
    print('------Finished------')
    return model


EMB_SIZE = 64
NEG_WEIGHT = 0.1
DROP_RATIO = 0.3
LEARNING_RATE = 0.01
BATCH_SIZE = 32
EPOCH = 600
model = train(train_od, train_time, train_y, test_od, test_time, test_y, epoch=EPOCH, batch_size=BATCH_SIZE, emb_size=EMB_SIZE, lr=LEARNING_RATE, 
              drop_out=DROP_RATIO, neg_weight=NEG_WEIGHT, n_origin=N_ORIGIN, n_destination=N_DESTINATION, n_time=N_TIME)

ENMF(
  (origin_embs): Embedding(12, 64)
  (destination_embs): Embedding(12, 64)
  (time_embs): Embedding(48385, 64)
  (dropout): Dropout(p=0.3, inplace=False)
)
------第1个epoch------
train_loss: -0.027771917037640315
------第2个epoch------
train_loss: -1.1112172704190015
------第3个epoch------
train_loss: -2.6974470373243093
------第4个epoch------
train_loss: -4.613855094462633
------第5个epoch------
train_loss: -6.841515807807445
------第6个epoch------
train_loss: -7.405362331867218
------第7个epoch------
train_loss: -14.470906600356102
------第8个epoch------
train_loss: -19.90074529647827
------第9个epoch------
train_loss: -15.206150156259536
------第10个epoch------
train_loss: -25.32704668045044
------第11个epoch------
train_loss: -37.05662040114403
------第12个epoch------
train_loss: -22.651872754096985
------第13个epoch------
train_loss: -42.976387155056
------第14个epoch------
train_loss: -52.52664442062378
------第15个epoch------
train_loss: -59.07671785354614
------第16个epoch------
train_loss: -32.38325783

train_loss: -24127.494928741457
------第299个epoch------
train_loss: -28629.13584485054
------第300个epoch------
train_loss: -22844.461190986633
------第301个epoch------
train_loss: -25160.280409431456
------第302个epoch------
train_loss: -23762.57316467762
------第303个epoch------
train_loss: -26213.792184185982
------第304个epoch------
train_loss: -26747.613556671142
------第305个epoch------
train_loss: -24890.733438062667
------第306个epoch------
train_loss: -27716.112363624572
------第307个epoch------
train_loss: -25268.11728925705
------第308个epoch------
train_loss: -27529.980505561827
------第309个epoch------
train_loss: -26402.920419692993
------第310个epoch------
train_loss: -25017.7019361496
------第311个epoch------
train_loss: -25791.93531165123
------第312个epoch------
train_loss: -24951.451989078523
------第313个epoch------
train_loss: -24862.75510072708
------第314个epoch------
train_loss: -33122.52952589989
------第315个epoch------
train_loss: -25910.69784787893
------第316个epoch------
train_loss: -26982.

train_loss: -101789.15950276851
------第598个epoch------
train_loss: -90715.41377182007
------第599个epoch------
train_loss: -91312.49447607994
------第600个epoch------
train_loss: -92276.63786315918
24.316550420032744
24.412554268614162
RMSE: 0.011300988716456389  ≈ 0.011301
MAE : 0.0031216838179277057  ≈ 0.003122
TER : 0.9960674394196906  ≈ 0.996067
------Finished------


In [8]:
torch.cuda.empty_cache()

In [9]:
#dataset = np.loadtxt(fname='E:/data/Abilene.txt', delimiter="\t", skiprows=1)

In [10]:
#train_index = np.random.choice(list(range(dataset.shape[0])), size = int(0.7*dataset.shape[0]), replace=False)
#test_index = list(set(range(dataset.shape[0])) - set(train_index))

In [11]:
#np.savetxt('E:/data/Abilene_train.txt', dataset[train_index,:])
#np.savetxt('E:/data/Abilene_test.txt', dataset[test_index,:])